In [1]:
# Importing the common libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import re

# TensorFlow and Scikit Learn libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Create a function that imports the data and returns 6 dataframes for each of the 6 models
def importData():
    # Importing the data and convert to a GeoDF
    df = pd.read_csv(r'../../Data/GNIP/GNIP_Cleaned.csv')
    df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Lon, df.Lat, df.Alt)).set_crs('EPSG:4326')
    cols = df.columns

    # Remove units from columns for easier processing
    codeCols = list(map(lambda x: re.sub(r'\(([^()]*)\)', '', x).strip(), cols))
    df.columns = codeCols

    # Load in the PrevailingWinds file for which the model will be split on
    modelLocations = pd.read_csv(r'../../Data/ModelSplit_Arch/PrevailingWinds_6Split.csv')
    modelLocations = gpd.GeoDataFrame(modelLocations, geometry=gpd.GeoSeries.from_wkt(modelLocations['Geometry']), crs="EPSG:4326")
    modelLocations.drop(columns="Geometry", inplace=True)
    modelLocations.set_index('Prevailing Wind', inplace=True)

    # Create an empty dictionary that will contain each of the modelDatasets to train with
    modelData = {}
    for modelLoc in modelLocations.index:
        modelData[modelLoc] = df[df.within(modelLocations.loc[modelLoc].geometry)]

    return df, cols, modelLocations

In [ ]:
# Main function to call all the other functions
def main():
    # Importing the data
    df, cols, test = importData()
    return df, cols, test

In [ ]:
df, null, test = main()

/tmp/ipykernel_2324/3266678229.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  test.area


Prevailing Wind
PolarWesterlies_NW     9780.0
Trade_W                9780.0
PolarWesterlies_SW     9780.0
PolarWesterlies_NE    11820.0
Trade_E               11820.0
PolarWesterlies_SE    11820.0
dtype: float64